In [1]:
#import libraries 

import pandas as pd 
import numpy as np 
import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes  
import folium 

import time 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

! pip install geocoder
import geocoder

from bs4 import BeautifulSoup

print ('Libraries Imported')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /Users/monicagonzalez/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-2.0.0          | 63 KB     | ##################################### | 100% 
Prepari

In [2]:
List = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List).text

In [3]:
soup = BeautifulSoup(source, 'xml')

In [4]:
table=soup.find('table')

In [5]:
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [6]:
df

,Postalcode,Borough,Neighborhood


In [7]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [8]:
df

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [9]:
df = df[~df['Borough'].isnull()]  # to filter out bad rows
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
df['Neighborhood'].replace('Not assigned',df['Borough'],inplace=True)
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [10]:
from geopy.util import get_version
get_version()

'2.0.0'

In [11]:
geolocator = Nominatim(scheme='http', user_agent="foursquare_agent")


for row_index, item in df.iterrows():
    
    list1 = df.loc[[row_index],['Neighborhood']].values.astype('str')
    loc = ' , Toronto, Ontario, Canada'
    list1.astype('str')
    list1 = np.append(list1, loc)
    latitude = None
    longitude = None
    location = None
    
    location = geolocator.geocode(list1 , limit = 15)
    time.sleep(5)
    if(location is not None):
        df.loc[df.index[row_index], 'Latitude'] = location.latitude
        df.loc[df.index[row_index], 'Longitude'] = location.longitude
          

In [12]:
print(df)

    Postalcode      Borough  \
0          M1B  Scarborough   
1          M1C  Scarborough   
2          M1E  Scarborough   
3          M1G  Scarborough   
4          M1H  Scarborough   
..         ...          ...   
98         M9N         York   
99         M9P    Etobicoke   
100        M9R    Etobicoke   
101        M9V    Etobicoke   
102        M9W    Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                       Malvern, Rouge  43.809196 -79.221701  
1               Rouge Hill, Port Union, Highland Creek        NaN        NaN  
2                    Guildwood, Morningside, West Hill  43.753480 -79.179995  
3                                               Woburn  43.759824 -79.225291  
4                                            Cedarbrae  43.756467 -79.226692  
..                                                 ...        ...        ...  
98                                              Weston  43.700161 -79.5162

In [13]:

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [15]:
df

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.221701
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.753480,-79.179995
3,M1G,Scarborough,Woburn,43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,43.756467,-79.226692
...,...,...,...,...,...
98,M9N,York,Weston,43.700161,-79.516247
99,M9P,Etobicoke,Westmount,43.693640,-79.521043
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",NaN,NaN
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",NaN,NaN


In [16]:
df.dropna(inplace =True)
df.index = pd.RangeIndex(len(df.index))

In [17]:
df

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.221701
1,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.753480,-79.179995
2,M1G,Scarborough,Woburn,43.759824,-79.225291
3,M1H,Scarborough,Cedarbrae,43.756467,-79.226692
4,M1J,Scarborough,Scarborough Village,43.743742,-79.211632
5,M1S,Scarborough,Agincourt,43.785353,-79.278549
6,M1W,Scarborough,"Steeles West, L'Amoreaux West",43.798206,-79.317969
7,M1X,Scarborough,Upper Rouge,43.804930,-79.165837
8,M2H,North York,Hillcrest Village,43.681695,-79.425712
9,M2K,North York,Bayview Village,43.769197,-79.376662


In [18]:
map_df = folium.Map(location=[latitude, longitude], zoom_start=10)
#folium is a visualization library.

#add markers to map 
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_df)  
    
map_df

In [19]:
CLIENT_ID = 'KGUWPQJDWHJE0ONSH2IX2XINLE2YXTBJYDFGSV2SWSM0HIVJ' # your Foursquare ID
CLIENT_SECRET = '35T1QIU2AZK52ARZIZG0TEALI0PGSKVCKRXZDBH3MHC0VG3H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KGUWPQJDWHJE0ONSH2IX2XINLE2YXTBJYDFGSV2SWSM0HIVJ
CLIENT_SECRET:35T1QIU2AZK52ARZIZG0TEALI0PGSKVCKRXZDBH3MHC0VG3H


In [20]:
#let us explore the first neighborhood
df.loc[0, 'Neighborhood']

'Malvern, Rouge'

In [21]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.8091955, -79.2217008.


In [22]:
#let's create the GET request URL. Name your URL url.

LIMIT = 100 # limit of number of venues returned by Foursquare AP

radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KGUWPQJDWHJE0ONSH2IX2XINLE2YXTBJYDFGSV2SWSM0HIVJ&client_secret=35T1QIU2AZK52ARZIZG0TEALI0PGSKVCKRXZDBH3MHC0VG3H&v=20180605&ll=43.8091955,-79.2217008&radius=500&limit=100'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0f14c21d7260470b7a325c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 13,
  'suggestedBounds': {'ne': {'lat': 43.813695504500004,
    'lng': -79.21547671927222},
   'sw': {'lat': 43.8046954955, 'lng': -79.22792488072777}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '545ecf75498ec51130da75e0',
       'name': 'Shoppers Drug Mart',
       'location': {'address': '1400 Neilson Rd',
        'lat': 43.80961,
        'lng': -79.222729,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80961,
          'lng':

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
#clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/monicagonzalez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Shoppers Drug Mart,Pharmacy,43.809610,-79.222729
1,Subway,Sandwich Place,43.806961,-79.221476
2,Pizza Hut,Pizza Place,43.808326,-79.220616
3,Pizza Pizza,Pizza Place,43.806613,-79.221243
4,Shoppers Drug Mart,Pharmacy,43.806489,-79.223024


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

13 venues were returned by Foursquare.


In [27]:
#Explore neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
#create a new dataframe

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Malvern, Rouge
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Agincourt
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Bayview Village
York Mills, Silver Hills
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Downsview
Downsview
Downsview
Downsview
Victoria Village
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth West, Riverdale
Studio District
Lawrence Park
Davisville North
Davisville
Moore Park, Summerhill East
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
St. James Town
Berczy Park
Central Bay Street
Commerce Court, Victoria Hotel
Roselawn
University of Toronto, Harbord
Lawrence Manor, Lawrence Heights
Glencairn
Christie
Dufferin, Dovercourt Village
Runnymede, The Junction North
Parkdale, Roncesvalles
Runnymede, Swansea
Alderwood, Long Branch
Islington Avenue, Humber Valley Village
Humber Summit
Humberlea, Emery
Weston
Westmount


In [29]:
print(toronto_venues.shape)

(1737, 7)


In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,12,12,12,12,12,12
"Alderwood, Long Branch",8,8,8,8,8,8
Bayview Village,11,11,11,11,11,11
Berczy Park,100,100,100,100,100,100
Cedarbrae,27,27,27,27,27,27
Central Bay Street,100,100,100,100,100,100
Christie,59,59,59,59,59,59
Church and Wellesley,69,69,69,69,69,69
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


In [34]:
#Analyze each neighborhood

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
toronto_grouped =toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [36]:
toronto_grouped
#the mean of frequency of each category was taken 

,Neighborhood,Yoga Studio,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.083333,...,0.000000,0.00,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Berczy Park,0.010000,0.000000,0.000000,0.000000,0.010000,0.02000,0.00,0.000000,0.000000,...,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Cedarbrae,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Central Bay Street,0.010000,0.000000,0.000000,0.000000,0.000000,0.01000,0.01,0.000000,0.010000,...,0.000000,0.00,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.020000
6,Christie,0.000000,0.000000,0.000000,0.016949,0.000000,0.00000,0.00,0.016949,0.000000,...,0.000000,0.00,0.000000,0.000000,0.016949,0.016949,0.000000,0.016949,0.000000,0.000000
7,Church and Wellesley,0.028986,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.014493,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Commerce Court, Victoria Hotel",0.010000,0.000000,0.000000,0.030000,0.000000,0.02000,0.00,0.000000,0.000000,...,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
9,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [37]:
#Print each neighborhood with the top 10 most common venues 

num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                   venue  freq
0     Chinese Restaurant  0.25
1            Coffee Shop  0.08
2   Cantonese Restaurant  0.08
3      Korean Restaurant  0.08
4  Vietnamese Restaurant  0.08
5   Hong Kong Restaurant  0.08
6       Asian Restaurant  0.08
7          Shopping Mall  0.08
8          Train Station  0.08
9             Food Court  0.08


----Alderwood, Long Branch----
                       venue  freq
0                Pizza Place  0.25
1                        Pub  0.12
2                        Gym  0.12
3                   Pharmacy  0.12
4             Sandwich Place  0.12
5                Coffee Shop  0.12
6               Dance Studio  0.12
7  Latin American Restaurant  0.00
8      Performing Arts Venue  0.00
9                  Nightclub  0.00


----Bayview Village----
                  venue  freq
0                  Bank  0.18
1           Gas Station  0.09
2        Breakfast Spot  0.09
3  Fast Food Restaurant  0.09
4        Sandwich Place  0.09
5    Persian Rest

                  venue  freq
0  Fast Food Restaurant  0.15
1           Pizza Place  0.15
2              Pharmacy  0.15
3     Convenience Store  0.08
4         Grocery Store  0.08
5          Skating Rink  0.08
6        Sandwich Place  0.08
7                  Park  0.08
8            Restaurant  0.08
9       Bubble Tea Shop  0.08


----Moore Park, Summerhill East----
                     venue  freq
0                     Park   0.4
1               Playground   0.2
2              Candy Store   0.2
3            Grocery Store   0.2
4              Yoga Studio   0.0
5     Pakistani Restaurant   0.0
6              Music Store   0.0
7              Music Venue   0.0
8               Nail Salon   0.0
9  New American Restaurant   0.0


----Parkdale, Roncesvalles----
                         venue  freq
0                  Coffee Shop  0.10
1                     Pharmacy  0.10
2  Eastern European Restaurant  0.07
3                         Café  0.07
4                       Bakery  0.07
5             

In [38]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Chinese Restaurant,Food Court,Vietnamese Restaurant,Cantonese Restaurant,Shopping Mall
1,"Alderwood, Long Branch",Pizza Place,Dance Studio,Pub,Coffee Shop,Sandwich Place
2,Bayview Village,Bank,Sporting Goods Shop,Fast Food Restaurant,Pizza Place,Sandwich Place
3,Berczy Park,Coffee Shop,Café,Restaurant,Japanese Restaurant,Italian Restaurant
4,Cedarbrae,Fast Food Restaurant,Coffee Shop,Toy / Game Store,Furniture / Home Store,Clothing Store


In [42]:
#So far we know the top 5 venues in each neighborhood 
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Chinese Restaurant,Food Court,Vietnamese Restaurant,Cantonese Restaurant,Shopping Mall
1,"Alderwood, Long Branch",Pizza Place,Dance Studio,Pub,Coffee Shop,Sandwich Place
2,Bayview Village,Bank,Sporting Goods Shop,Fast Food Restaurant,Pizza Place,Sandwich Place
3,Berczy Park,Coffee Shop,Café,Restaurant,Japanese Restaurant,Italian Restaurant
4,Cedarbrae,Fast Food Restaurant,Coffee Shop,Toy / Game Store,Furniture / Home Store,Clothing Store
5,Central Bay Street,Coffee Shop,Café,Clothing Store,Italian Restaurant,Boutique
6,Christie,Korean Restaurant,Coffee Shop,Sandwich Place,Grocery Store,Mexican Restaurant
7,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant
8,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Restaurant,Café,Italian Restaurant
9,Davisville,Italian Restaurant,Sushi Restaurant,Coffee Shop,Furniture / Home Store,Pub


In [43]:
#Now we are going to cluster the neighborhoods 

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [48]:
toronto_merged=df
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.221701,3,Pizza Place,Fast Food Restaurant,Pharmacy,Sandwich Place,Grocery Store
1,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.753480,-79.179995,4,Park,Women's Store,Ethiopian Restaurant,Fountain,Food Truck
2,M1G,Scarborough,Woburn,43.759824,-79.225291,3,Fast Food Restaurant,Coffee Shop,Discount Store,Clothing Store,Bank
3,M1H,Scarborough,Cedarbrae,43.756467,-79.226692,3,Fast Food Restaurant,Coffee Shop,Toy / Game Store,Furniture / Home Store,Clothing Store
4,M1J,Scarborough,Scarborough Village,43.743742,-79.211632,3,Coffee Shop,Pharmacy,Chinese Restaurant,Fast Food Restaurant,Gym


In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
#now let us examine the clusters and determine the discriminating venue category that distinguish each cluster 

#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,East York,0,Skating Rink,Dance Studio,Athletics & Sports,Bus Stop,Pharmacy
24,East York,0,Sandwich Place,Japanese Restaurant,Convenience Store,Park,Filipino Restaurant
31,Central Toronto,0,Park,Grocery Store,Playground,Candy Store,Women's Store
32,Downtown Toronto,0,Park,Food Truck,Playground,Bike Trail,Women's Store
51,North York,0,Park,Bakery,Construction & Landscaping,Women's Store,Fish & Chips Shop
52,North York,0,Nightclub,Golf Course,Park,Coffee Shop,Women's Store
54,Etobicoke,0,Pizza Place,Gas Station,Park,Women's Store,Electronics Store


In [51]:
#Cluster 2

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
50,Etobicoke,1,Playground,Women's Store,Ethiopian Restaurant,Fountain,Food Truck


In [52]:
#Cluster 3 

toronto_merged.loc[tornto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,North York,2,Pool,Irish Pub,Fountain,Food Truck,Food Court


In [53]:
#Cluster 4

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Scarborough,3,Pizza Place,Fast Food Restaurant,Pharmacy,Sandwich Place,Grocery Store
2,Scarborough,3,Fast Food Restaurant,Coffee Shop,Discount Store,Clothing Store,Bank
3,Scarborough,3,Fast Food Restaurant,Coffee Shop,Toy / Game Store,Furniture / Home Store,Clothing Store
4,Scarborough,3,Coffee Shop,Pharmacy,Chinese Restaurant,Fast Food Restaurant,Gym
5,Scarborough,3,Chinese Restaurant,Food Court,Vietnamese Restaurant,Cantonese Restaurant,Shopping Mall
6,Scarborough,3,Fast Food Restaurant,Chinese Restaurant,Supermarket,Breakfast Spot,Coffee Shop
8,North York,3,Italian Restaurant,Coffee Shop,Indian Restaurant,Café,Mexican Restaurant
9,North York,3,Bank,Sporting Goods Shop,Fast Food Restaurant,Pizza Place,Sandwich Place
11,North York,3,Bubble Tea Shop,Korean Restaurant,Pizza Place,Ramen Restaurant,Dessert Shop
12,North York,3,Coffee Shop,Restaurant,Pub,Sandwich Place,Gym


In [54]:
#Cluster 5

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Scarborough,4,Park,Women's Store,Ethiopian Restaurant,Fountain,Food Truck
7,Scarborough,4,Fast Food Restaurant,Park,Women's Store,Ethiopian Restaurant,Fountain
